In [73]:
#!pip install hanlp
#!pip install simalign
! spacy download it_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 32.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_md')


In [1]:
import os, csv
# public
import hanlp
import pandas as pd
from tqdm import tqdm
from simalign import SentenceAligner
# from google.cloud import translate_v2 as translate

/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [20]:
# target 
language = "Spanish"

RESOURCE = 'res'
RESULTS = os.path.join(RESOURCE, 'results')
TOKENS_FILE = os.path.join(RESULTS, 'tokens-'+language+'.tsv')
TRANS_FILE = os.path.join(RESULTS, 'sentences-'+language+'-Translations.tsv')
KEY = os.path.join(RESOURCE, 'key')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'tonal-works-420505-eda807c7cc52.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS


#DATA

In [21]:
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()
sids = trans_df["ID"].tolist()

In [22]:
tgt_sents = trans_df[language+' Translation'].tolist()


# TOKENIZATION

In [23]:
import spacy
nlp = spacy.load("es_core_news_md")
# nlp = spacy.load("it_core_news_md")


In [24]:
tk_ids, tks_list, pos_list , lemma_list = [], [], [], []
total = 0
x = []
for sid, tgt_s in zip(sids, tgt_sents):
    doc = nlp(tgt_s)
    seq_len = len(doc)
    total += seq_len
    for token in doc:
        tks_list.append(token.text)
        pos_list.append(token.pos_)
        lemma_list.append(token.lemma_)
    tk_ids += [f'{sid}.t{i:03}' for i in range(1, seq_len+1)]
len(tk_ids), len(tks_list), len(pos_list), len(lemma_list)
#print(total)

(2787, 2787, 2787, 2787)

In [25]:
len(lemma_list)

2787

In [26]:
tks_df = pd.DataFrame({
    'Token ID': tk_ids
    , 'Token': tks_list
    , 'Lemma': lemma_list
    , 'POS': pos_list
    })
tks_df.head()

,Token ID,Token,Lemma,POS
0,es001.s001.t001,Este,este,DET
1,es001.s001.t002,documento,documento,NOUN
2,es001.s001.t003,es,ser,AUX
3,es001.s001.t004,un,uno,DET
4,es001.s001.t005,resumen,resumen,NOUN


In [27]:
tks_df.to_csv(TOKENS_FILE, sep='\t', index=False)